In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

## Data Fetching

In [2]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_16120\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [3]:
data

,Spindle Speed,Feed (mm/min),Tool Dia. (mm),Depth of Cut (mm),Length of Cut (mm),Length of cut actual(mm),Width of Cut(mm),Actual Width(mm),Wi,Wf,delta Weight,Time (sec),Time (min),MRR ideal,MRR actual,Density(g/mm3),Delta L,Delta W
0,3000,10,1,0.3,11,11.048,1,1.187,49.3753,49.3694,0.0059,121.35,2.0225,3,1.076450814,0.00271,0.048,0.187
1,3000,15,1,0.3,11,11.064,1,1.002,49.3694,49.3682,0.0012,81.2,1.353333333,4.5,0.3271953902,0.00271,0.064,0.002
2,3000,20,1,0.3,11,11.05,1,1.145,49.3682,49.3659,0.0023,63.13,1.052166667,6,0.806629324,0.00271,0.05,0.145
3,3250,10,1,0.3,11,11.0565,1,1.14425,49.3659,49.3635,0.0024,110.825,1.847833333,3,1.007537102,0.00271,0.0565,0.14425
4,3250,15,1,0.3,11,11.0605,1,1.08075,49.3635,49.3617,0.0018,91.185,1.51975,4.5,0.5536889701,0.00271,0.0605,0.08075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,4750,15,2,0.5,12,12.185,2,2.081,47.2419,47.2106,0.0313,95.87,1.597833333,15,10.82681853,0.00271,0.185,0.081
104,4750,20,2,0.5,12,12.166,2,2.063,47.2106,47.1822,0.0284,66.05,1.100833333,20,12.08555939,0.00271,0.166,0.063
105,5000,10,2,0.5,12,12.139,2,2.076,47.1822,47.1535,0.0287,124.65,2.0775,10,6.269954396,0.00271,0.139,0.076
106,5000,15,2,0.5,12,12.139,2,2.125,47.1535,47.1256,0.0279,83.28,1.388,15,9.038995289,0.00271,0.139,0.125


## Data Preprocessing

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [5]:
X_train = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train=pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [7]:
X_train.shape

(86, 16)

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[86,16]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(2, activation="relu")
])
model.compile(loss="mean_squared_error", optimizer="sgd")

In [11]:
model.fit(X_train,y_train)

ValueError: in user code:

    File "f:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "f:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "f:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "f:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "f:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "f:\Proje